In [2]:
import math
import numpy as np
import sys

def getbin(radius,angle,region_size):

    max_radius=np.max(radius)
   
    # print(radius[0])
    bin=[]
    # bin=np.array((20,4))
    # temp=[]
    for m in range(20):
        row=[]
        for n in range(4):
            row.append([])
        bin.append(row)
        # temp.append(row)
  
    
   
    for m in range(20): #theta
        theta_low=m*18
        theta_up=(m+1)*18
        for n in range(4): #radius
            rho_low=max_radius*n/4
            rho_up=max_radius*(n+1)/4
            temp=[]
            num=0
            
            for row in range(region_size[0]):
                for col in range(region_size[1]):
                
                    # if (radius[row] [col] >= rho_low) and (radius[row][col] <= rho_up) and (angle[row][col] >= theta_low) and (angle[row][col] <= theta_up):
                        temp.append((row,col))
                      
                  
            bin[m][n]=temp
            
            # print(bin)
            
    # print(len(bin[0][0]))
    # f=open("test.txt","a")
    # f.write(str(bin))
    # f.close()
    # exit()
    
    return bin
   
    

In [3]:
import math
import numpy as np

def cart2polar(region_size):  #cartesian to polar
    radius=np.zeros((region_size[0],region_size[1]))
    angle=np.zeros((region_size[0],region_size[1]))
    center=[math.ceil(region_size[0]/2),math.ceil(region_size[1]/2)]
    for row in range(region_size[0]):
        for col in range(region_size[1]):
            x=row-center[0]
            y=col-center[1]
            rho=np.sqrt((x**2)+(y**2))
            theta=np.arctan2(y, x)
            
            epsilon = 1e-8
            if(rho!=0):
                radius[row][col]=np.log(rho)
            else:
                radius[row][col]=(epsilon)
            angle[row][col]=(theta*(180/math.pi)+180)
            
    return radius,angle      


In [46]:

from sklearn.preprocessing import MinMaxScaler
from scipy.io import savemat
import cv2

def call_ssd(patch, region, alpha, center_patch):
    # patch_size=patch_size.shape
    region_size = region.shape

    SSD_region = np.zeros((region_size[0], region_size[1]))

    for row in range(center_patch[0]+1, region_size[1]-center_patch[1]):
        for col in range(center_patch[1], region_size[1]-center_patch[1]):
            temp = region[row-center_patch[0]:center_patch[0] +
                          row, col-center_patch[1]:center_patch[1]+col, :]

            diff = temp-patch
            SSD_region[row][col] = np.sum(diff**2)
            SSD_region[row][col] = np.exp(-alpha*SSD_region[row][col])
  
    return SSD_region


def get_self_sim_vec(SSD_region, bin, vec_size):
    self_similarities_vec = np.zeros((vec_size))
    
    
  

    num = 0
    for m in range(20):
        for n in range(4):
            temp = bin[m][n]
            
            max_value = 0
            temp_size = len(temp)

            for loc in range(temp_size):
                row, col = temp[loc]

                max_value = max(max_value, SSD_region[row][col])

            self_similarities_vec[num] = max_value
            num = num+1
    return self_similarities_vec


def dr(image, sig_score_img, region_size, scale):
    
    ma =np.max(np.max(sig_score_img))
    mi=np.min(np.min(sig_score_img))
    norm_sig_score_img = (sig_score_img - mi)/(ma-mi)
    norm_sig_score_img = norm_sig_score_img*255
    out = cv2.resize(norm_sig_score_img, (2,2))
    print(out[0])

    
    # cv2.imshow("Resized (Height) ", resized)
    # cv2.waitKey(0)
   
    
    # print(norm_sig_score_img[0][42])
    # # position = find_element_2d(norm_sig_score_img, ma)
    # print((ma))
    # # print(sig_score_img[0][42])
    
    # print(position)
    # print(norm_sig_score_img.shape)
    
    
    # color = (255, 0, 0)
    # thickness = 5
    


# Find the element

    

    # image = cv2.rectangle(image, start_point, end_point, color, thickness)
    # cv2.imshow("f", image) 
    # # cv2.destroyAllWindows()
    # cv2.waitKey(0)


def com_Self_Similarities(imgRgb, region_size, patch_size, bin):
    lab_image = cv2.cvtColor(imgRgb, cv2.COLOR_BGR2LAB)

    lab_size = lab_image.shape
    vec_size = 80
    alpha = 1/(85**2)
    self_similarities = np.zeros((lab_image.shape[0], lab_image.shape[1], vec_size))
    # print(lab_image.shape,"s")
    center_region = [math.floor(region_size[0]/2),
                     math.floor(region_size[1]/2)]
    center_patch = [math.floor(patch_size[0]/2), math.floor(patch_size[1]/2)]
    # print(center_patch)

    for row in range(center_region[0], lab_size[0] - center_region[0]):
        # print(f" {row}>{count}")
        for col in range(center_region[1], lab_size[1]-center_region[1]):
            
            
            

            
            
            patch = lab_image[row-center_patch[0]:row + center_patch[0],
                              col-center_patch[0]:col + center_patch[0], :]
            region = lab_image[row - center_region[0]:row + center_region[0] + 1,
                               col - center_region[1]:col + center_region[1] + 1, :]
            
            # print(f"patch{row-center_patch[0]}:{row + center_patch[0] } >> {col-center_patch[0]}:{col + center_patch[0]}")
            print(f"{row - center_region[0]}:{row + center_region[0] } >> {col - center_region[1]}:{col + center_region[1] }")
            # print(region.shape,'region')
            # print(patch.size)
            
        
            SSD_region = call_ssd(patch, region, alpha, center_patch)
       
            
            vec = get_self_sim_vec(SSD_region, bin, vec_size)
            # print(vec.shape)
            vec = vec.reshape(-1, 1)

            scaler = MinMaxScaler(feature_range=(0, 1))
            scaler.fit_transform(vec)
            vec = vec.flatten()
            # print(vec)

            self_similarities[row, col, :] = vec
            # print(self_similarities.shape)
            # self_similarities[r]
            # print(a)

    return self_similarities

In [47]:

region_size = [46, 37]
patch_size = [5, 5]

radius, angle = cart2polar(region_size)
# print(radius)

bin = getbin(radius, angle, region_size)


In [48]:
import sys
# n_img=2
# for i in range(n_img):

#     name = f'Input{i+1}.jpg'
#     # print(name)
#     image = cv2.imread(f'{name}', 1)
#     # plt.imshow(image)
#     # plt.show()
#     # exit()

    
    
#     imgRgb=cv2.resize(image, (0, 0), fx = 1/3, fy = 1/3)

#     self_similarities=com_Self_Similarities(imgRgb,region_size,patch_size,bin)
#     # print(self_similarities)
#     np.save(f'mat{i+1}', self_similarities)




from PIL import Image
import matplotlib.pyplot
import matplotlib.pyplot as plt
import matplotlib.image as img
from main import n_img


image1 = cv2.imread('1.jpg')
image2 = cv2.imread("2.jpg")

imgRgb1=cv2.resize(image1, (0, 0), fx = 1/5, fy = 1/5)
imgRgb2 = cv2.resize(image2, (0, 0), fx = 1/5, fy = 1/5)
# imgRgb1=cv2.resize(image1, (80, 80))
# imgRgb2=cv2.resize(image2, (80, 80))

print(imgRgb1.shape,'img1', imgRgb2.shape,'img2')
# cv2.imshow("lalala", imgRgb1) 
# cv2.waitKey(0)




self_similarities1 = com_Self_Similarities(imgRgb1, region_size, patch_size, bin)
# print(self_similarities1)

# self_similarities2 = com_Self_Similarities(imgRgb2, region_size, patch_size, bin)
# print(self_similarities2)





(48, 72, 3) img1 (48, 71, 3) img2
0:46 >> 0:36
0:46 >> 1:37
0:46 >> 2:38
0:46 >> 3:39
0:46 >> 4:40
0:46 >> 5:41
0:46 >> 6:42
0:46 >> 7:43
0:46 >> 8:44
0:46 >> 9:45
0:46 >> 10:46
0:46 >> 11:47
0:46 >> 12:48
0:46 >> 13:49
0:46 >> 14:50
0:46 >> 15:51
0:46 >> 16:52
0:46 >> 17:53
0:46 >> 18:54
0:46 >> 19:55
0:46 >> 20:56
0:46 >> 21:57
0:46 >> 22:58
0:46 >> 23:59
0:46 >> 24:60
0:46 >> 25:61
0:46 >> 26:62
0:46 >> 27:63
0:46 >> 28:64
0:46 >> 29:65
0:46 >> 30:66
0:46 >> 31:67
0:46 >> 32:68
0:46 >> 33:69
0:46 >> 34:70
0:46 >> 35:71
1:47 >> 0:36
1:47 >> 1:37
1:47 >> 2:38
1:47 >> 3:39
1:47 >> 4:40
1:47 >> 5:41
1:47 >> 6:42
1:47 >> 7:43
1:47 >> 8:44
1:47 >> 9:45
1:47 >> 10:46
1:47 >> 11:47
1:47 >> 12:48
1:47 >> 13:49
1:47 >> 14:50
1:47 >> 15:51
1:47 >> 16:52
1:47 >> 17:53
1:47 >> 18:54
1:47 >> 19:55
1:47 >> 20:56
1:47 >> 21:57
1:47 >> 22:58
1:47 >> 23:59
1:47 >> 24:60
1:47 >> 25:61
1:47 >> 26:62
1:47 >> 27:63
1:47 >> 28:64
1:47 >> 29:65
1:47 >> 30:66
1:47 >> 31:67
1:47 >> 32:68
1:47 >> 33:69
1:47 >

In [209]:


width = 1
height = 1
center_sub = [math.floor(width/2), math.floor(height/2)]
p = 1
img_size1 = self_similarities1.shape
# print(center_sub)
# print(imgRgb1.shape)
# print(self_similarities1.shape,"s")
max_match = {}
match_score={}
sig_score_img = np.zeros((img_size1[0], img_size1[1]))
for row1 in range(center_sub[0], img_size1[0]-center_sub[0]):
    for col1 in range(center_sub[1], img_size1[1]-center_sub[1]):
        # print(row1,col1)
        sub1 = self_similarities1[row1 - center_sub[0]:row1 +center_sub[0]+1, col1 - center_sub[1]:col1 + center_sub[1]+1, :]
        temp1 = np.tile(sub1, (self_similarities2.shape[0], self_similarities2.shape[1], 1))
        temp2 = -1 * np.sum((self_similarities2 - temp1) ** 2, axis=2)
        max_match[1] = np.max(np.max(temp2))
        # print(max_match)
        match_score[1] = (temp2)
        # print(match_score)
        avgMatch=0
        stdMatch=0
        sum_max_match=0
        avgMatch+=np.mean(temp2)
        stdMatch+=np.std(temp2)
        sum_max_match+=max_match[1]
        sig_score_img[row1][col1] = (sum_max_match-avgMatch)/stdMatch
        # print(sig_score_img)
        # print(row1,col1)
        
    

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# import numpy as np
# ma = np.max(sig_score_img)
# mi = np.min(sig_score_img)
# norm_sig_score_img = (sig_score_img - mi) / (ma - mi)  # Normalize to range [0, 1]
# norm_sig_score_img = norm_sig_score_img * 255  # Scale to range [0, 255]
# rect_img = np.zeros((sig_score_img.shape[0],sig_score_img.shape[1]))
# def find_element_2d(arr, value):
#     return np.unravel_index(np.argmax(arr), arr.shape)

# # Find the position of the maximum value
# position = find_element_2d(sig_score_img, ma)
# print("Position of max value:", position)




# start_x, start_y = position
# end_x = start_x + region_size[0]/2
# end_y = start_y + region_size[1]/2

# end_x = min(end_x, img.shape[1]/2)
# end_y = min(end_y, img.shape[0]/2)

# norm_sig_score_img = cv2.resize(norm_sig_score_img, (0, 0), fx = 3, fy = 3,interpolation = cv2.INTER_LINEAR)

# # cv2.rectangle(imgRgb2, (start_x, start_y), (end_x, end_y), color=(255, 0, 0), thickness=2)


# cv2.imshow("Result", norm_sig_score_img)
# cv2.imwrite("my.png",img)
# # # cv2.waitKey(0)
# # # cv2.destroyAllWindows()


In [204]:

ma =np.max(np.max(sig_score_img))
mi=np.min(np.min(sig_score_img))
norm_sig_score_img = (sig_score_img - mi)/(ma-mi)
norm_sig_score_img = norm_sig_score_img*255



# print(norm_sig_score_img.shape)
# x = find_element_2d(sig_score_img, ma)
x, y = np.where(sig_score_img == ma)
print(x)
# x,y=x.item(),y.item()
# print(x) 
# print(y)
# x,y=1,2
# norm_sig_score_img = cv2.resize(norm_sig_score_img, (0, 0), fx = 3, fy = 3)

# plt.figure()
# plt.imshow(norm_sig_score_img)
# plt.show()
# rect_img = np.zeros((sig_score_img.shape[0],sig_score_img.shape[1]))



# cv2.rectangle(img, (position[0], position[1]), (position[0]+region_size[0],position[0]+ region_size[0]), color=(255,0,0), thickness=2)
# rect_img[x-math.floor(region_size[0]/2):x+math.floor(region_size[0]/2),y-math.floor(region_size[1]/2):y+math.floor(region_size[1]/2)]=128

# print(rect_img)
#

# rect_img = cv2.resize(norm_sig_score_img, (0, 0), fx = 3, fy = 3,interpolation = cv2.INTER_LINEAR)
# rect_size=[imgRgb1.shape[0], imgRgb1.shape[1]]


# src_img = np.copy(imgRgb1)
# # src_img[:, :, 1] = np.clip(src_img[:, :, 1] + rect_size, 0, 255)

# # print(src_img)
# # cv2.imwrite("my.png",rect_img)

# plt.figure()
# plt.imshow(rect_img)
# plt.show()
# k = cv2.waitKey(0)

[24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24]


[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
